# Example Initial Pipeline for Saliency Object Detection based on Eye Tracking Data

In [ ]:
import os
if not os.path.exists('sam3'):
    !git clone https://github.com/facebookresearch/sam3.git

In [ ]:
!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

sample_data


In [ ]:
%cd sam3

In [ ]:
!pip install -e ".[notebooks]"

In [3]:
from huggingface_hub import login
login()

In [ ]:
import torch
#################################### For Image ####################################
from PIL import Image
import numpy as np
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor
# Load the model
model = build_sam3_image_model(enable_inst_interactivity=True)
processor = Sam3Processor(model)
# Load an image

In [ ]:
import os
from fixation_centroids import *

fixation -> centroids points for prompting model


In [ ]:
participant = 'P01'
path = '/content/drive/MyDrive/Colab Notebooks/my_projects/sam/fixations'
fixation_data = {}

for file in os.listdir(path):
    if file.endswith('.parquet') and file.startswith(participant):
        if not file.startswith("P01_IMG001"):
          continue
        full_path = os.path.join(path, file)
        centroids = calculate_fixation_centroids(full_path)
        image_name = file.replace(f'{participant}_', '').replace('.parquet', '.jpg')

        # Store data in dictionary
        if image_name not in fixation_data:
            fixation_data[image_name] = {
                'participants': [],
                'centroids': []
            }

        fixation_data[image_name]['participants'].append(participant)
        fixation_data[image_name]['centroids'].append(centroids)
fixation_data

overlay mask for detection

In [ ]:
import numpy as np
import matplotlib

def overlay_masks(image, masks):
    image = image.convert("RGBA")

    # Handle different mask shapes - squeeze out batch and channel dimensions if present
    if len(masks.shape) == 4:
        masks = masks.squeeze(0)  # Remove batch dimension
    if len(masks.shape) == 3 and masks.shape[0] == 1:
        masks = masks.squeeze(0)  # Remove channel dimension if it's 1

    # Convert to numpy and scale to 0-255
    masks_np = 255 * masks.astype(np.uint8)

    # If we have a single mask, add a dimension
    if len(masks_np.shape) == 2:
        masks_np = masks_np[np.newaxis, ...]

    n_masks = masks_np.shape[0]
    cmap = matplotlib.colormaps.get_cmap("rainbow").resampled(n_masks)
    colors = [
        tuple(int(c * 255) for c in cmap(i)[:3])
        for i in range(n_masks)
    ]

    for mask, color in zip(masks_np, colors):
        mask = Image.fromarray(mask)
        overlay = Image.new("RGBA", image.size, color + (0,))
        alpha = mask.point(lambda v: int(v * 0.5))
        overlay.putalpha(alpha)
        image = Image.alpha_composite(image, overlay)
    return image



predict and store the original image with the mask overlay

In [ ]:
masks= None
scores=None
logits= None
image = None

for image_filename in fixation_data:
  nr_images +=1
  full_image_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/my_projects/sam/img_bin',image_filename)


  image = Image.open(full_image_path)
  width, height = image.size
  inference_state = processor.set_image(image)
  nr_centroids = 0
  for centroid in fixation_data[image_filename].get("centroids")[0]:# not multiple Participant yet so specify 0 works
    print(centroid)
    pixel_x = round(centroid[0] * width)
    pixel_y = round(centroid[1] * height)
    point = [[pixel_x, pixel_y]]

    label = np.array([1])

    masks, scores, logits = model.predict_inst(
    inference_state,
    point_coords=point,
    point_labels=label,
    multimask_output=False,
    )

    sorted_ind = np.argsort(scores)[::-1]
    masks = masks[sorted_ind]
    scores = scores[sorted_ind]
    logits = logits[sorted_ind]

    annotated_img = overlay_masks(image.copy(), masks=masks)

   # Save the annotated image
    path_example_img = '/content/drive/MyDrive/Colab Notebooks/my_projects/sam/example_outputs'
    # Create filename using base image name + participant + centroid counter
    base_name = os.path.splitext(image_filename)[0]  # Remove .jpg extension
    output_filename = f"{base_name}_participant0_centroid{nr_centroids}.png"
    output_path = os.path.join(path_example_img, output_filename)

    annotated_img.save(output_path, 'PNG')
    nr_centroids +=1